In [1]:
import os, sys
os.environ["SPARK_HOME"] = '/usr/hdp/current/spark-client'
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], 'python'))

# import package
import pyspark
from pyspark.context import SparkContext, SparkConf

conf = SparkConf().set("spark.ui.port", 61604)
#.setMaster('yarn-client')
sc = SparkContext(conf=conf)

In [175]:
sc.stop()

In [2]:
from dateutil import parser

In [12]:
#hdfs://backend-0-3.insight.gsu.edu:8029/user/gmaheshwari1
#all_csv = sc.textFile("/data/MSA_8050_Spring_18/proj04/yellow*.csv").map(lambda x: x.split(","))
NY = sc.textFile("/data/MSA_8050_Spring_18/proj04/yellow*.csv").map(lambda x: x.split(","))

In [14]:
NY_nohead  = NY.filter(lambda x: str(x[1]) != "tpep_pickup_datetime")

In [42]:
#parser.parse(NY_nohead.take(4)[0][2]).weekday()

In [8]:
#NY_nohead.take(5)

In [93]:
#NY.take(4)
#header = NY.first()
#NY = NY.filter(lambda row: row!= header)
header

[u'VendorID',
 u'tpep_pickup_datetime',
 u'tpep_dropoff_datetime',
 u'passenger_count',
 u'trip_distance',
 u'pickup_longitude',
 u'pickup_latitude',
 u'RatecodeID',
 u'store_and_fwd_flag',
 u'dropoff_longitude',
 u'dropoff_latitude',
 u'payment_type',
 u'fare_amount',
 u'extra',
 u'mta_tax',
 u'tip_amount',
 u'tolls_amount',
 u'improvement_surcharge',
 u'total_amount']

In [ ]:
# Data prep

In [17]:
# Day,hour, (tip amount, 1)
NY1  = NY_nohead.map(lambda x : (parser.parse(str(x[1])), (float(x[15]),1)))
#Makinf data by hour and day
NY_DH = NY1.map(lambda x : ((x[0].weekday(), x[0].hour), x[1]))
#NY_H = NY1.map(lambda x : (( x[0].hour), x[1]))
#NY_D = NY1.map(lambda x : (( x[0].weekday()), x[1]))
#NY_D.take(4)

In [188]:
#NY_DH_red.take(4)

In [118]:
NY_DH.map(lambda x : (x[1][1],1)).reduceByKey(lambda x, y: x+y).collect()
#.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).collect()

[(1, 34499859)]

# Average tip per ride - total tip/number of rides

In [18]:
#grouping by - day hour : total tip, total cust
NY_DH_red  = NY_DH.reduceByKey(lambda x,y: (float(x[0])+float(y[0]), x[1]+y[1]))
#NY_H_red  =  NY_H.reduceByKey(lambda x,y: (float(x[0])+float(y[0]), x[1]+y[1]))
#NY_D_red  =  NY_D.reduceByKey(lambda x,y: (float(x[0])+float(y[0]), x[1]+y[1]))

In [29]:
#Day, hour : tip total and number of rides
NY_DH_red.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()
#sig

[((5, 7), 1.7160115555668167),
 ((4, 6), 1.87847684266601),
 ((0, 8), 1.8163223371371915),
 ((1, 4), 1.9002235533630878),
 ((6, 5), 1.794604692514528),
 ((3, 15), 1.9009119849083358),
 ((2, 4), 1.9172164214523022),
 ((4, 8), 1.769569596274818),
 ((3, 8), 1.8036418860182981),
 ((3, 18), 1.836448078229965),
 ((2, 11), 1.7737806199869994),
 ((0, 19), 1.7539184545974698),
 ((4, 3), 1.6082232684749267),
 ((1, 11), 1.756872963447348),
 ((6, 10), 1.4377516906851333),
 ((5, 5), 2.0692598935854387),
 ((0, 10), 1.74749719020546),
 ((6, 17), 1.7722733366396481),
 ((2, 1), 1.772669639873903),
 ((4, 10), 1.6860423962064712),
 ((0, 4), 1.9815077214404646),
 ((5, 11), 1.3961908625323285),
 ((0, 21), 1.9568838423367478),
 ((0, 15), 1.69980256569419),
 ((1, 16), 1.8877081331414642),
 ((3, 20), 1.9283187462076954),
 ((4, 13), 1.714487067891021),
 ((4, 23), 1.8817164609453516),
 ((5, 0), 1.8334393595692549),
 ((2, 14), 1.8313583607681228),
 ((6, 8), 1.5376329707095582),
 ((1, 9), 1.7951714994537573),
 ((

In [26]:
NY_DH_red.take(204)

[((5, 7), (135432.77999999988, 78923)),
 ((4, 6), (243313.47000000026, 129527)),
 ((0, 8), (451870.1200000019, 248783)),
 ((1, 4), (48535.50999999999, 25542)),
 ((6, 5), (82759.98999999998, 46116)),
 ((3, 15), (483677.5500000015, 254445)),
 ((2, 4), (56554.05000000001, 29498)),
 ((4, 8), (468948.33000000066, 265007)),
 ((3, 8), (518709.3700000023, 287590)),
 ((3, 18), (610725.5000000007, 332558)),
 ((2, 11), (409122.5000000014, 230650)),
 ((0, 19), (511988.090000002, 291911)),
 ((4, 3), (115796.90000000014, 72003)),
 ((1, 11), (396289.05000000104, 225565)),
 ((6, 10), (285733.02000000066, 198736)),
 ((5, 5), (90614.95999999995, 43791)),
 ((0, 10), (371603.53000000084, 212649)),
 ((6, 17), (457409.57000000007, 258092)),
 ((2, 1), (122025.25999999986, 68837)),
 ((4, 10), (377961.81000000087, 224171)),
 ((0, 4), (60691.59999999999, 30629)),
 ((5, 11), (330411.3600000006, 236652)),
 ((0, 21), (492549.62000000174, 251701)),
 ((0, 15), (410757.290000001, 241650)),
 ((1, 16), (402717.99000000

In [10]:
# ##Day wise average tipe
#NY_D_red.map(lambda x: (x[0], x[1][0]/x[1][1])).takeOrdered(7, key = lambda x: -x[1])

In [11]:
###Hour wise average tipe
#NY_H_red.map(lambda x: (x[0], x[1][0]/x[1][1])).takeOrdered(6, key = lambda x: x[1])

# Percentage of people giving tip  - rides having tip/number of rides

In [ ]:
# Day hour
import numpy as np
NY_DH_fraction_giving_tip  = NY_DH.map(lambda x: (x[0], (np.count_nonzero(x[1][0]), x[1][1])))
NY_DH_fraction_giving_tip_1 = NY_DH_fraction_giving_tip.reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))
#NY_DH_fraction_giving_tip_1.take(4)

In [ ]:
NY_DH_fraction_giving_tip_2 = NY_DH_fraction_giving_tip_1.map(lambda x: (x[0], (1.0*x[1][0]/x[1][1])))

In [28]:
###sig
NY_DH_fraction_giving_tip_2.take(200)

[((5, 7), 0.5591652623443103),
 ((4, 6), 0.6351957506929057),
 ((0, 8), 0.7020294795062364),
 ((1, 4), 0.47502153316106804),
 ((6, 5), 0.4806357880128372),
 ((3, 15), 0.6084026017410442),
 ((2, 4), 0.4830835988880602),
 ((4, 8), 0.71353964234907),
 ((3, 8), 0.7242358913731354),
 ((3, 18), 0.6702439875149598),
 ((2, 11), 0.6237979622805117),
 ((0, 19), 0.6776997098430686),
 ((4, 3), 0.5393108620474147),
 ((1, 11), 0.6161195220889766),
 ((6, 10), 0.5931235407777151),
 ((5, 5), 0.4921787581923226),
 ((0, 10), 0.6186532737045554),
 ((6, 17), 0.6247229670040141),
 ((2, 1), 0.5977599256213955),
 ((4, 10), 0.6341364404851654),
 ((0, 4), 0.47281987658754776),
 ((5, 11), 0.6072841133816743),
 ((0, 21), 0.6951303332128199),
 ((0, 15), 0.5915332091868405),
 ((1, 16), 0.6075739323230382),
 ((3, 20), 0.7091059774163473),
 ((4, 13), 0.6011826678272667),
 ((4, 23), 0.6641332167039233),
 ((5, 0), 0.6675771248277311),
 ((2, 14), 0.613360807977744),
 ((6, 8), 0.5738703074509315),
 ((1, 9), 0.68966982277

# Tip percentage by total amount

In [132]:
#Date, tip, total amount
NY_tip_bytotal = NY_nohead.map(lambda x : (parser.parse(x[1]), (float(x[15]),float(x[18]))))
NY_fil_tip = NY_tip_bytotal.filter(lambda x: x[1][0]>0)
NY_tip_bytotal_1 = NY_fil_tip.map(lambda x : ((x[0].weekday(), x[0].hour), (x[1][0], x[1][1])))

In [133]:
#NY_fil = NY1.filter(lambda x: x[1][0]>0)
#NY_tip_bytotal_2.take(4)

In [134]:
NY_tip_bytotal_2 = NY_tip_bytotal_1.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
NY_tip_bytotal_3 = NY_tip_bytotal_2.map(lambda x: (x[0], x[1][0]/x[1][1]))
#NY_tip_bytotal_3.collect()

In [135]:
NY_tip_bytotal_3.takeOrdered(10, key = lambda x: x[1])

[((1, 6), 0.1563011117235425),
 ((0, 6), 0.15635477435809653),
 ((0, 7), 0.15700633813299614),
 ((2, 6), 0.15710977417892644),
 ((1, 7), 0.1572504946884164),
 ((5, 20), 0.15728402083005713),
 ((3, 7), 0.1573895150358136),
 ((3, 6), 0.15744516737731265),
 ((4, 7), 0.15749725065475687),
 ((5, 23), 0.15750749356321278)]

# Tip percentage by Speed

In [ ]:
#% of people gave the tip

In [141]:
NY_speed = NY_nohead.filter(lambda x: (parser.parse(x[2])-parser.parse(x[1])).total_seconds()>0).map(lambda x: ((float(x[4])*3600/(parser.parse(x[2])-parser.parse(x[1])).total_seconds()), (np.count_nonzero(float(x[15])),1)))
#NY_speed.takeOrdered(10, key = lambda x: -x[0])

In [142]:
NY_speed.take(4)

[(17.9027027027027, (0, 1)),
 (16.710280373831775, (0, 1)),
 (6.050420168067227, (0, 1)),
 (32.142857142857146, (0, 1))]

In [150]:
bins = np.array([10, 20, 30, 40, 50,60,70,80,90,100,120,130])
NY_speed1 = NY_speed.filter(lambda x: x[0]< 130).filter(lambda x: x[0] > 0)
NY_speed2 = NY_speed1.map(lambda x: (bins[np.digitize(x[0], bins)], x[1] ))
NY_speed3 = NY_speed2.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))
#NY_speed3.take(4)

In [43]:
NY_speed2.take(4)

NameError: name 'NY_speed2' is not defined

In [159]:
#NY_speed2.reduceByKey(lambda x,y: x + y).take(4)

In [37]:
parser.parse("2018-04-30").weekday()

0